In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# mute warning

In [2]:
import pandas as pd
import numpy as np
import recordlinkage as rl

###  1. Data preprocess
#### 1.1 Sample is from result_rl_for_preds in Gary's 20200107_Record-linkage, 6.1. Processing data for predictions

In [3]:
sample = pd.read_csv('sample.csv') #Based on Sorted Neighbourhood
sample = sample.set_index(['id1', 'id2'])

In [4]:
sample

,,first_name,last_name,state,race,county,place,place_of_birth,sex,birth_year
id1,id2,,,,,,,,,
0,41902,0.0,0.0,0.0,1,0.0,1.0,0.0,0,0.0
1,50171,1.0,0.0,1.0,0,1.0,1.0,0.0,1,0.0
2,37255,1.0,0.0,1.0,1,0.0,1.0,1.0,1,0.0
3,2887,1.0,0.0,0.0,0,0.0,0.0,1.0,1,0.0
4,39779,1.0,0.0,0.0,0,0.0,0.0,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...
50840,45632,1.0,0.0,0.0,1,1.0,1.0,0.0,1,0.0
50841,8584,1.0,0.0,0.0,0,0.0,1.0,0.0,1,0.0
50842,36355,1.0,0.0,0.0,0,0.0,0.0,0.0,1,0.0


In [5]:
sample.describe().T

,count,mean,std,min,25%,50%,75%,max
first_name,256520.0,0.974142,0.158711,0.0,1.000000e+00,1.000000e+00,1.0,1.0
last_name,256520.0,0.580130,0.493538,0.0,0.000000e+00,1.000000e+00,1.0,1.0
state,256520.0,0.322002,0.467245,0.0,0.000000e+00,0.000000e+00,1.0,1.0
race,256520.0,0.585713,0.492600,0.0,0.000000e+00,1.000000e+00,1.0,1.0
county,256520.0,0.294246,0.455704,0.0,0.000000e+00,0.000000e+00,1.0,1.0
place,256520.0,0.592036,0.491457,0.0,0.000000e+00,1.000000e+00,1.0,1.0
place_of_birth,256520.0,0.228485,0.419858,0.0,0.000000e+00,0.000000e+00,0.0,1.0
sex,256520.0,0.863438,0.343386,0.0,1.000000e+00,1.000000e+00,1.0,1.0
birth_year,256520.0,0.273333,0.436636,0.0,9.828413e-237,4.484155e-44,1.0,1.0


#### 1.2 Train_X is the first 5k pairs in the sample

In [6]:
train_X = sample[0:5000]

#### 1.3 Train_Y is the manual mark pairs in 1860-1930 database corresponding to the 5k pairs in sample

In [7]:
df = pd.read_excel("1860-1930 database for family tracking Adam manual AFAM CANADA current.xlsx")
df.head()

,CalculatedBirthYear,ID,Census.Year,State/Province,County,Place,Last.Name,First.Name,Age,Sex,...,Enlistment Place,Date Mustered Out,Year of this Record,Unnamed: 41,Unnamed: 42,Last Name MATCH,First Name Match,Census Year Match,Unnamed: 46,Total of Matches
0,1757,NaN,1850,LA,NaN,New Orleans,Jeffries,?Ucky,93,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1757,358,1860,MN,PINE,CHENGWATANA,LUSSENE,JOSEPH,103,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1759,NaN,1850,NJ,NaN,Newark,Freeman,Flora,91,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1759,NaN,1864,CanadaWest,Essex,Windsor,LYNCH,MINTIE,105,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1760,NaN,1850,NaN,NaN,NaN,Gorett,Margaret,90,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
train_Y = []
for index, row in train_X.iterrows():
    index1 = index[0] 
    index2 = index[1]
    if(df.loc[index1,'Notable'] == 0): continue
    if(df.loc[index1,'Notable'] == 'nan'): continue
    if(df.loc[index1,'Notable'] == df.loc[index2,'Notable']):
        train_Y.append(index)
print(len(train_Y))
print(train_Y[:4])

10
[(9813, 9817), (417, 512), (491, 492), (22815, 685)]


In [9]:
temp=list(map(list, zip(*train_Y)))

In [10]:
train_Y=pd.MultiIndex.from_arrays(temp,names=['id1', 'id2'])

###  2. Supervised Learning - Naive Bayes Classifier.

#### binarize=0.35

In [11]:
# Train the classifier
nb = rl.NaiveBayesClassifier(binarize=0.35)
nb.fit(train_X, train_Y)

# Predict the match status for all record pairs
result_nb = nb.predict(sample)

len(result_nb)

5080

#### Remove matches already marked in 1860-1930 database

In [12]:
detect = []
for i in range(len(result_nb)):
    index1 = result_nb[i][0]
    index2 = result_nb[i][1]
    if(index1==index2): continue
    elif(df.loc[index1,'Notable']== df.loc[index2,'Notable']): pass
    else: 
        detect.append(result_nb[i])
print(len(detect))
#detect

234


In [13]:
detect

[(2465, 2467),
 (8260, 8771),
 (3718, 3719),
 (3730, 3509),
 (6499, 6500),
 (17602, 16983),
 (26452, 26455),
 (35237, 35238),
 (17119, 17120),
 (17120, 17119),
 (172, 173),
 (173, 172),
 (17561, 18673),
 (18673, 17561),
 (349, 350),
 (350, 349),
 (6402, 6755),
 (6755, 6402),
 (16443, 17153),
 (17153, 16443),
 (19765, 21527),
 (21527, 19765),
 (552, 553),
 (553, 552),
 (627, 628),
 (628, 627),
 (873, 874),
 (874, 873),
 (989, 990),
 (990, 989),
 (11889, 11890),
 (11890, 11889),
 (1317, 1318),
 (1318, 1317),
 (1346, 1347),
 (1347, 1346),
 (12203, 13621),
 (13621, 12203),
 (26704, 26705),
 (26705, 26704),
 (1815, 1816),
 (1816, 1815),
 (1836, 1837),
 (1837, 1836),
 (2092, 2093),
 (2093, 2092),
 (2102, 2103),
 (2103, 2102),
 (23889, 23890),
 (23890, 23889),
 (2296, 2297),
 (2297, 2296),
 (2435, 2436),
 (2436, 2435),
 (2596, 2597),
 (2597, 2596),
 (3064, 3065),
 (3065, 3064),
 (3130, 3131),
 (3131, 3130),
 (3137, 3138),
 (3138, 3137),
 (3201, 3202),
 (3202, 3201),
 (3390, 3391),
 (3391, 339

In [14]:
for i in range(len(detect)):
    idx1 = detect[i][0]
    idx2 = detect[i][1]
    print(df.loc[idx1]["First.Name"],df.loc[idx1]["Last.Name"],df.loc[idx1]["Age"], df.loc[idx1]["Sex"],df.loc[idx1]["Place"],df.loc[idx1]["Notable"])
    print(df.loc[idx2]["First.Name"],df.loc[idx2]["Last.Name"],df.loc[idx2]["Age"], df.loc[idx2]["Sex"],df.loc[idx2]["Place"],df.loc[idx2]["Notable"])
    print("\n")
    

William Burk  42 M Windsor YYY WDSR
William Burke 62 M Anderdon YY WDSR


Elizabeth Johnson 26 F Windsor YY WDSR
Elizabeth Johnston 25 nan Township of Chatham and Gore nan


ROBERT STARK 47 M DETROIT 33828000
ROBERT STARKS 57 M DETROIT 4422107000


LOUISA TOMLINSON 57 F DETROIT 4422106000
LOUISA TOMPSON 48 F DETROIT 33828000


Charles Davidson 30 nan District Two nan
Charles Davis 30 nan District Two nan


WILLIAM TOMLINSON 31 M DETROIT 4422106000
WILLIAM TOMPSON 22 M DETROIT 33828000


MARY TOMLINSON 22 F DETROIT 4422106000
MARY TOMPSON 12 F DETROIT 33828000


MARY STARK 1 F DETROIT 33828000
MARY STARKS 11 F DETROIT 4422107000


Rutha Brown 12 nan Township of Chatham and Gore nan
Rutha Brown 12 nan Township of Chatham and Gore nan


Rutha Brown 12 nan Township of Chatham and Gore nan
Rutha Brown 12 nan Township of Chatham and Gore nan


Mary Burley 70 nan Township of Chatham and Gore nan
Mary Burley 70 nan Township of Chatham and Gore nan


Mary Burley 70 nan Township of Chatham and G

R Dorsey 4 M St. Andrew's Ward nan
R Dorsey 4 M St. Andrew's Ward nan


James Rider 30 nan Township of Chatham and Gore nan
James Rider 30 nan Township of Chatham and Gore nan


James Rider 30 nan Township of Chatham and Gore nan
James Rider 30 nan Township of Chatham and Gore nan


George Stanton 10 nan St. George's Ward nan
George Stanton 8 nan St. George's Ward nan


George Stanton 8 nan St. George's Ward nan
George Stanton 10 nan St. George's Ward nan


C Cooper 28 M St. Andrew's Ward nan
C Cooper 28 M St. Andrew's Ward nan


C Cooper 28 M St. Andrew's Ward nan
C Cooper 28 M St. Andrew's Ward nan


H Atkinson 27 M St. George's Ward nan
H Atkinson 27 M St. George's Ward nan


H Atkinson 27 M St. George's Ward nan
H Atkinson 27 M St. George's Ward nan


W E Barnum 27 M St. Andrew's Ward nan
W E Barnum 27 M St. Andrew's Ward nan


W E Barnum 27 M St. Andrew's Ward nan
W E Barnum 27 M St. Andrew's Ward nan


V Howard 27 F St. Andrew's Ward nan
V Howard 27 F St. Andrew's Ward nan


V Ho

M Dorsey 3 F St. Andrew's Ward nan
M Dorsey 3 F St. Andrew's Ward nan


M Howard 3 F St. Andrew's Ward nan
M Howard 3 F St. Andrew's Ward nan


M Howard 3 F St. Andrew's Ward nan
M Howard 3 F St. Andrew's Ward nan


RICHARD MOORE 22 M DETROIT 4417560000
RICHARD MOORE 22 M DETROIT 4422719000


RICHARD MOORE 22 M DETROIT 4422719000
RICHARD MOORE 22 M DETROIT 4417560000


Emma Hardy 2 nan Township of Chatham and Gore nan
Emma Hardy 2 nan Township of Chatham and Gore nan


Emma Hardy 2 nan Township of Chatham and Gore nan
Emma Hardy 2 nan Township of Chatham and Gore nan


J G Meads 2 F St. George's Ward nan
J G Meads 2 F St. George's Ward nan


J G Meads 2 F St. George's Ward nan
J G Meads 2 F St. George's Ward nan


Harriett J Brown 1 nan Township of Chatham and Gore nan
Harriett J Brown 1 nan Township of Chatham and Gore nan


Harriett J Brown 1 nan Township of Chatham and Gore nan
Harriett J Brown 1 nan Township of Chatham and Gore nan


W Howard 1 M St. Andrew's Ward nan
W Howard 1 M 

In [15]:
result =[] 
for i in range(len(detect)):
    idx1 = detect[i][0]
    idx2 = detect[i][1]
    result.append(df.loc[idx1])
    result.append(df.loc[idx2])
    
result

[CalculatedBirthYear                                            1819
 ID                                                              NaN
 Census.Year                                                    1861
 State/Province                                              Ontario
 County                                                        Essex
 Place                                                       Windsor
 Last.Name                                                     Burk 
 First.Name                                                  William
 Age                                                              42
 Sex                                                               M
 Color..Race.or.Ethnicity                                      Black
 lat                                                             NaN
 long                                                            NaN
 address                                                         NaN
 MARITAL                          

In [16]:
len(result)

468

In [17]:
result = pd.DataFrame(result)
result.head()

,CalculatedBirthYear,ID,Census.Year,State/Province,County,Place,Last.Name,First.Name,Age,Sex,...,Enlistment Place,Date Mustered Out,Year of this Record,Unnamed: 41,Unnamed: 42,Last Name MATCH,First Name Match,Census Year Match,Unnamed: 46,Total of Matches
2465,1819,NaN,1861,Ontario,Essex,Windsor,Burk,William,42,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,1819,Matson,1881,Ontario,Essex,Anderdon,Burke,William,62,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8260,1835,NaN,1861,Ontario,Essex,Windsor,Johnson,Elizabeth,26,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8771,1836,NaN,1861,Ontario,Kent,Township of Chatham and Gore,Johnston,Elizabeth,25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3718,1823,1870 IPUMS 1% sample,1870,MI,WAYNE,DETROIT,STARK,ROBERT,47,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
import itertools
lst = range(0,int(result.shape[0]/2))
result['matched_pair']=list(itertools.chain.from_iterable(itertools.repeat(x, 2) for x in lst))

In [19]:
result.head()

,CalculatedBirthYear,ID,Census.Year,State/Province,County,Place,Last.Name,First.Name,Age,Sex,...,Date Mustered Out,Year of this Record,Unnamed: 41,Unnamed: 42,Last Name MATCH,First Name Match,Census Year Match,Unnamed: 46,Total of Matches,matched_pair
2465,1819,NaN,1861,Ontario,Essex,Windsor,Burk,William,42,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2467,1819,Matson,1881,Ontario,Essex,Anderdon,Burke,William,62,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8260,1835,NaN,1861,Ontario,Essex,Windsor,Johnson,Elizabeth,26,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8771,1836,NaN,1861,Ontario,Kent,Township of Chatham and Gore,Johnston,Elizabeth,25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3718,1823,1870 IPUMS 1% sample,1870,MI,WAYNE,DETROIT,STARK,ROBERT,47,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [22]:
result.to_csv('Yiwei_NaiveBayes.csv')